In [1]:
import pandas as pd

In [2]:
import numpy as np

In [44]:
columns = ['data', 'area', 'number_product', 'balances', 'element']
list_operfas = []
for i in os.listdir("operfas"):
    i = pd.read_csv('operfas/'+i, sep='\s+', encoding='CP1251', header = None)
    
    list_operfas.append(i)


for i in list_operfas:
    i.dropna(axis=1, inplace=True)
    i.dropna(axis=0, how='all', inplace=True)
    i.columns = columns
    i['balances'] = i['balances'].apply(lambda x: float(x.split()[0].replace(',', '.')))
    

    
req_3_factory = pd.DataFrame(pd.concat(list_operfas, ignore_index=True))

req_3_factory

req_3_factory.drop(['data', 'element'], axis=1, inplace=True)

fiz_ostatki = req_3_factory.groupby(['area','number_product'], as_index=False).sum()

scrap = pd.read_excel("брак/брак.xlsx",usecols=[0,2,3],names=['number_product','balances', 'area'])

scrap = scrap.reindex(columns=['area','number_product', 'balances'])

scrap['balances'] = scrap['balances']*1000
scrap['area'] = 198349
pd.merge(scrap, fiz_ostatki, on=['area','number_product'])
temp_requirements = pd.merge(fiz_ostatki, scrap, on=['area','number_product'], how='left', suffixes=('_req', '_scrap'))
temp_requirements['balances_scrap'].fillna(0, inplace=True)
temp_requirements['balances'] = temp_requirements['balances_req']-temp_requirements['balances_scrap']
fiz_ostatki = temp_requirements.drop(['balances_scrap', 'balances_req'], axis=1)

def area_change_name(area):
    if area == 1078:
        return 'Волгоградский НПЗ'
    elif area == 198349:
        return 'Пермнефтеоргсинтез ООО'
    elif area == 119388:
        return 'Тюменский Филиал'
    else:
        return area
    
fiz_ostatki['area'] = fiz_ostatki.apply(lambda x: area_change_name(x['area']), axis=1)
fiz_ostatki['balances'] = fiz_ostatki['balances']/1000

In [45]:
orders

,Код КССС,Пункт отгрузки,"осталось произвести, тн"
0,1779443,ВНП,10
1,23,ПНОС,3


In [46]:
graphic

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
0,ВНП,1779443,2022-06-11,8
1,ВНП,1779443,2022-06-12,5
2,ВНП,1779443,2022-06-13,3


In [11]:
graphic['Объем_фасовки, т'].apply(np.sum)

0     8
1     9
2    12
3    64
4     2
Name: Объем_фасовки, т, dtype: int32

In [12]:
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
df

,A,B
0,4,9
1,4,9
2,4,9


In [20]:
def sqrt():
    return 26

In [27]:
df.apply(lambda x: sqrt())

A    26
B    26
dtype: int64

In [243]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Дата_производства'].dt.date.iloc[0]

datetime.date(2022, 6, 10)

In [82]:
graphic[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465)]

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
0,ПНОС,207465,2022-06-10,9.0
1,ПНОС,207465,2022-06-11,15.0


In [18]:
len_of_dates = len(graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465)])

In [22]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Объем_фасовки, т'][0]

9.0

In [23]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Объем_фасовки, т'][1]

15.0

In [230]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465) & (graphic['Дата_производства']=='2022-06-10'), 'Объем_фасовки, т']

0    0.0
Name: Объем_фасовки, т, dtype: float64

In [268]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465) & 
                        (graphic['Дата_производства']=='2022-06-10'), 'Объем_фасовки, т'] 

0    20.0
Name: Объем_фасовки, т, dtype: float64

In [297]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Дата_производства'].iloc[1]), 'Дата_производства'].iloc[0]

Timestamp('2022-06-11 00:00:00')

In [282]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Дата_производства'].iloc[0]

Timestamp('2022-06-10 00:00:00')

In [355]:
def if_volume_less_than_required(city, code, required_to_make):
    print('объем меньше')
       # print(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)])
        ## и если возвращает несколько дат, то
    if len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) > 1:

        ## вводим переменную длины, чтобы не выйти за пределы
        len_of_dates = len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)])
        ## вводим счетчик для того чтобы присвоить правильную дату
        i = 0
        ## создаем накопительную переменную 
        storage_var = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0]


        ## пока объем фасовки меньше, чем требуется
        while storage_var<required_to_make and len_of_dates>1:
            print('в цикле')
            len_of_dates-=1
            ## переходим к следующей
            i+=1
            ## и прибавляем к накопительной переменной
            storage_var+= graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
             ## и обнуляем в графике объем фасовки
            graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
            print('storage fresh, ', storage_var)
            print('required fresh, ' , required_to_make) 

            ## возьмем дату следующего производства
            data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                         (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                                                                    & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i]),
                                         'Дата_производства'].iloc[0]


            print('обнуляем')
            graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'] = 0

        print('data making', data_of_making)    
        ## вычитаем
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                    (graphic['Дата_производства']==data_of_making), 'Объем_фасовки, т'] = storage_var-required_to_make


        ## по окончанию цикла возвращает тот i который требуется    
        data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[i] 
        print(graphic)
        return data

In [356]:
def data_shipment(city, code, required_to_make):
    ## проверяем есть ли вообще в списке graphic
    if graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)].empty:
        return '-'
    ## если в таблице orders не требуется, то мы ставим черточку
    elif required_to_make == 0:
        return '-'
    ## иначе если есть и объема в графике достаточно, то мы    
    elif graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0] >= required_to_make:   
        print('Объем больше')
        print('graphic: ', graphic)
        ## вычитаем из объема
        ## добавить определенное дату, так как будет удалять все значения где код и пункт совпадаем
        data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                    (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                    & (graphic['Код КССС']==code), 'Дата_производства'].iloc[0]),'Дата_производства'].iloc[0]
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & (graphic['Дата_производства']==data_of_making),
                    'Объем_фасовки, т'] -= required_to_make
        # и возвращаем дату
        data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0]
        return data
    ## иначе если недостаточно объема чем требуется ...
    else:
        if_volume_less_than_required(city, code, required_to_make)
        
        ## иначе если возвращается только одна дата, то мы должны обнулить ее и присвоить дату
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'] = 0
        print(graphic)
        return graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0] 
        


In [357]:
orders['дата производства'] = orders.apply(lambda x: data_shipment(x['Пункт отгрузки'], x['КССС мат'], x['осталось произвести, тн']), axis=1)

объем меньше
в цикле
storage fresh,  24.0
required fresh,  11
обнуляем
data making 2022-06-11 00:00:00
  Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0           ПНОС    207465        2022-06-10              0.00
1           ПНОС    207465        2022-06-11             13.00
2           ПНОС    207464        2022-06-12             18.00
3            ВНП   3149286        2022-06-15             15.00
4            ВНП    196256        2022-06-15              1.33
5            ВНП    207465        2022-06-15             14.00
6         Тюмень   3148630        2022-06-11              5.00
7         Тюмень   3149286        2022-06-11             10.00
8         Тюмень    196256        2022-06-10              3.00
  Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0           ПНОС    207465        2022-06-10              0.00
1           ПНОС    207465        2022-06-11              0.00
2           ПНОС    207464        2022-06-12             18.00
3            ВН

In [358]:
orders

,КССС мат,Пункт отгрузки,"осталось произвести, тн",дата производства
0,207465,ПНОС,11,2022-06-10
1,207464,ПНОС,20,2022-06-12
2,207465,ПНОС,4,2022-06-10
3,3149286,ВНП,1,2022-06-15
4,196256,ВНП,2,2022-06-15
5,207465,ВНП,0,-
6,3148630,Тюмень,6,2022-06-11
7,3149286,Тюмень,8,2022-06-11
8,196256,Тюмень,9,2022-06-10


In [359]:
graphic

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
0,ПНОС,207465,2022-06-10,0.0
1,ПНОС,207465,2022-06-11,0.0
2,ПНОС,207464,2022-06-12,0.0
3,ВНП,3149286,2022-06-15,14.0
4,ВНП,196256,2022-06-15,0.0
5,ВНП,207465,2022-06-15,14.0
6,Тюмень,3148630,2022-06-11,0.0
7,Тюмень,3149286,2022-06-11,2.0
8,Тюмень,196256,2022-06-10,0.0


In [65]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Объем_фасовки, т']

0     9.0
1    15.0
Name: Объем_фасовки, т, dtype: float64

In [66]:
graphic.loc[(graphic['Пункт отгрузки']=='ВНП') & (graphic['Код КССС']==3149286), 'Объем_фасовки, т'].iloc[0]

3    15.0
Name: Объем_фасовки, т, dtype: float64

In [37]:
graphic[(graphic['Пункт отгрузки']=="ПНОС") & (graphic['Код КССС']==20746)].empty

True

In [127]:
merge_orders = orders.merge(graphic, on=['Пункт отгрузки', 'Код КССС'], how='inner')
merge_orders

,Код КССС,Пункт отгрузки,"осталось произвести, тн",Дата_производства,"Объем_фасовки, т"
0,1779443,ВНП,10,2022-06-11,8
1,1779443,ВНП,10,2022-06-12,5
2,1779443,ВНП,10,2022-06-13,3


In [113]:
merge_orders['осталось произвести, тн']

0    10
1    10
2    10
Name: осталось произвести, тн, dtype: int64

In [123]:
def data_distr(residuals):    
#если осталось произвести меньше или равно, то смотрим следующую строку
    if merge_orders['осталось произвести, тн'] <= merge_orders['Объем_фасовки, т']:
        #проверяем тот же ли код и пункт отгрузки, иначе возвращает предыдущую дату
        return 10
        


In [129]:
merge_orders.apply(lambda x: x['Код КССС'],x['Пункт отгрузки'],x['осталось произвести, тн']
                   x['Дата_производства'],x['Объем_фасовки, т'], axis=1)

0    1779443
1    1779443
2    1779443
dtype: int64

In [75]:
# merge_orders['difference'] = merge_orders['осталось произвести, тн'] - merge_orders['Объем_фасовки, т']
# merge_orders

In [79]:
merge_orders = merge_orders.set_index(['Пункт отгрузки','Код КССС'])

In [82]:
merge_orders.iloc[1]

осталось произвести, тн                     10
Дата_производства          2022-06-12 00:00:00
Объем_фасовки, т                             5
Name: (ВНП, 1779443), dtype: object

In [80]:
merge_orders

осталось произвести, тн Дата_производства  \
Пункт отгрузки Код КССС                                              
ВНП            1779443                        10        2022-06-11   
               1779443                        10        2022-06-12   
               1779443                        10        2022-06-13   

                         Объем_фасовки, т  
Пункт отгрузки Код КССС                    
ВНП            1779443                  8  
               1779443                  5  
               1779443                  3